In [17]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from tqdm import tqdm
import json

In [7]:
auth_file = "/Users/ie5/adversarial_faces/azure_auth.json"
with open(auth_file, "r") as f:
    auth_data = json.loads(f.read())


In [8]:
face_client = FaceClient(
    auth_data["endpoint"], 
    CognitiveServicesCredentials(auth_data["key"])
)

In [9]:
person_id_to_name = {}
identities = []
for p in face_client.person_group_person.list(person_group_id="community_naive_mean_10_5_0p1"):
    person_id_to_name[p.person_id] = p.name
    identities.append(p.name)

In [15]:
discovery = []
for protector in tqdm(identities):
    # We are sourcing query photos from epsilon_0.0.
    # In those cases, all subfolders in the "protected" identity have the same, clean
    # photo of the protector, so we just pick any single one that exists (e.g. n000958)
    # For the case where n000958 is itself the protector, n000958 is not present in its protected
    # subfolders, so we pick n000029 without loss of generality.
    if protector == "n000958":
        protected = "n000029"
    else:
        protected = "n000958"
        
    query_photos_paths = sorted(glob.glob(
        f"/Users/ie5/azureface/decoys/{protector}/community_naive_mean/{protected}/epsilon_0.0/png/*"
    ))
    
    # For this person group, we picked the first 10 lexicographically sorted
    # photos to use in the lookup set, so for the query, we use the 11th and on.
    # (The query is not supposed to be in the lookup set).
    for i in range(11, len(query_photos_paths)):
        faces_in_query_photos = face_client.face.detect_with_stream(
            open(query_photos_paths[i], "r+b"), 
            detectionModel='detection_02'
        )
        if len(faces_in_query_photos) > 0:
            break
        
    
    # There should only be one face, so we use that as the query face.
    results = face_client.face.identify(
        [faces_in_query_photos[0].face_id],
        "community_naive_mean_10_5_0p1"
    )
    
    if len(results) < 1 or len(results[0].candidates) < 1:
        discovery.append(1.0)
    
    else: 
        top_identity = person_id_to_name[results[0].candidates[0].person_id]
    
        # Note the switch of the term protector here:
        # protectors are also protected but we call them protectors because of the folder structure
        # In this case, the query photo belongs to the protector -- who is also protected by decoys 
        # of *other* protectors. Therefore, if the identity returned is that of the "protector,"
        # this is a failure in the defense.
        if top_identity == protector:
            discovery.append(1.0)
        else:
            discovery.append(0.0)
            
    time.sleep(10)
    

100%|██████████| 19/19 [03:16<00:00, 10.36s/it]


In [16]:
print(sum(discovery)/len(discovery))

1.0
